# SETUP PROJECT 1

Importeer pandas voor DataFrames en pyodbc voor database-connecties:

In [21]:
import sqlite3
import pandas as pd
import pyodbc

aenc_conn = sqlite3.connect("../../../Data/Raw/Sqlite/aenc.sqlite")
adventureworks_conn = sqlite3.connect("../../../Data/Raw/Sqlite/AdventureWorks.sqlite")

Database Connectie:

In [22]:
DB_SDM = {"servername": r"143.177.235.181", "database": "DEDSp1", "username": "sa", "password": "Str0ngP@ssword"}
DB_NorthWind = {"servername": r"143.177.235.181", "database": "NorthWind", "username": "sa", "password": "Str0ngP@ssword"} #Bestaat nog niet in DB : Moet nog toegevoegd worden...

export_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_SDM['servername']};"
    f"DATABASE={DB_SDM['database']};"
    f"UID={DB_SDM['username']};"
    f"PWD={DB_SDM['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

NorthWind_conn = pyodbc.connect( #Zou moeten werken indien het in database wordt toegevoegd.
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_NorthWind['servername']};"
    f"DATABASE={DB_SDM['database']};"
    f"UID={DB_SDM['username']};"
    f"PWD={DB_SDM['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

DataFrames maken voor tables:

In [24]:
def create_dataframes_sql(connection, db_type):
    dictionary : dict = {}
    query : str = ""
    key : str = ""

    if db_type == "sqlite":
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        key = "name"
    elif db_type == "ssms":
        query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
        key = "TABLE_NAME"

    # Try to read table names
    table_names = pd.read_sql(query, connection)

    for table in table_names[key].tolist():
        # Try to read each table, handle encoding issues
        try:
            # Read table data with a 'strict' option for decoding issues
            dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
        except Exception as e:
            print(f"Error reading {table}: {e}")
            continue

    return dictionary


aenc = create_dataframes_sql(aenc_conn, "sqlite")
adventureworks = create_dataframes_sql(adventureworks_conn, "sqlite")
northwind = create_dataframes_sql(NorthWind_conn, "ssms") #Zou moeten werken indien het in database wordt toegevoegd.

Error reading HumanResources_Employee: Could not decode to UTF-8 column 'OrganizationNode' with text 'Z�'
Error reading Person_Address: Could not decode to UTF-8 column 'SpatialLocation' with text '�'
Error reading sysdiagrams: Could not decode to UTF-8 column 'definition' with text '������'


/var/folders/k4/8yvf4x0n4c15r7pl4lwy8x6m0000gp/T/ipykernel_27578/3236788600.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_names = pd.read_sql(query, connection)


Dictionaries met DataFrames mergen naar één Dictionary:

In [34]:
sdm = aenc | adventureworks | northwind #Alle drie mergen in één Dictionary met alle DataFrames

sdm["HumanResources_Department"]

sdm['Department'] = sdm['Department'].rename(columns={"dept_id": "DepartmentID", "dept_name": "Name"})

df_combined = pd.concat([sdm["Department"], sdm["HumanResources_Department"]], ignore_index=True)
df_combined

#Hier komt veranderen van Employee en Department zodat het overeenkomt met Source Data Model

#dict_order = [] #Hierin komt dictionary order te staan.
                #Dit moet omdat de volgorde van welke tables worden gestuurd naar database belangrijk is.

#sdm = {k: sdm[k] for k in dict_order if k in sdm} #Verandert de volgorde van sdm-Dictionary.

,DepartmentID,Name,dept_head_id,GroupName,ModifiedDate
0,100,R & D,501.0,NaN,NaN
1,200,Sales,902.0,NaN,NaN
2,300,Finance,1293.0,NaN,NaN
3,400,Marketing,1576.0,NaN,NaN
4,500,Shipping,703.0,NaN,NaN
5,1,Engineering,NaN,Research and Development,04/30/18 00:00:00
6,2,Tool Design,NaN,Research and Development,04/30/18 00:00:00
7,3,Sales,NaN,Sales and Marketing,04/30/18 00:00:00
8,4,Marketing,NaN,Sales and Marketing,04/30/18 00:00:00
9,5,Purchasing,NaN,Inventory Management,04/30/18 00:00:00


SDM in SSMS vullen:

In [ ]:
export_cursor = export_conn.cursor()

for table_name, df in sdm.items():
    try:
        for index, row in df.iterrows():
            columns = df.columns.tolist()

            values = []
            for col in columns:
                value = row[col]

                if pd.isna(value):
                    values.append("NULL")

                elif isinstance(value, str):
                    values.append(f"'{value.replace("'", "''")}'")

                else:
                    values.append(str(value))

            column_names = ", ".join(columns)
            value_string = ", ".join(values)
            query = f"INSERT INTO {table_name} ({column_names}) VALUES ({value_string})"

            export_cursor.execute(query)
    except pyodbc.Error as e:
        print(f"Error in table: {table_name}")
        print(f"Query: {query}")
        print(f"Error message: {e}")
        print("-" * 80)

export_conn.commit()
export_cursor.close()